<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course5/week1/Dinosaurus_Island_Character_level_language_model_final_v3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character level language model - Dinosaurus Island

Dinosaurus Island에 오신 것을 환영합니다! 현실 세계에선 공룡은 6천 5백만년 전에 멸종했지만, 이번 과제에서는 그 공룡이 다시 돌아왔다고 합니다. 능력있는 생물학자들이 새로운 종류의 공룡을 만들어냈고, 당신의 임무는 이 공룡들에게 이름을 부여하는 것입니다. 공룡이 이름이 마음에 들지 않으면 미쳐 날뛸 수 있으니 현명하게 선택하세요!

<table>
<td>
<img src="arts/dino.jpg" style="width:250;height:300px;">
</td>
</table>


다행히도 여러분은 지금까지 딥 러닝을 배웠으며, 이를 사용하여 시간을 절약 할 수 있습니다. 여러분의 연구 보조원이 현존하는 모든 공룡 이름 목록을 수집하여 [dataset](dinos.txt)로 정리했습니다. (해당 링크를 클릭하여 자유롭게 살펴보십시오.) 새 공룡 이름을 만들려면 문자 수준의 언어 모델을 만들어 새 이름을 생성해야 합니다. 알고리즘은 위 공룡 이름 데이터를 바탕으로 패턴을 학습하고 무작위로 새 이름을 생성합니다. 여러분이 만들 알고리즘이 공룡의 분노로부터 여러분과 여러분의 팀을 안전하게 지켜 주길 바랍니다!

이 과제를 완료하면 다음을 배울 수 있습니다.

- RNN을 사용하여 처리 할 텍스트 데이터를 저장하는 방법
- 각 시간 단계에서 예측을 샘플링하여 다음 RNN-cell 단위로 전달하여 데이터를 합성하는 방법
- 문자 수준 텍스트 생성 순환 신경망 구축 방법
- 그래디언트 클리핑이 중요한 이유

`rnn_utils` 에서 제공 한 일부 함수를 로드하는 것으로 시작합니다. 특히, 이전 과제에서 구현 한 것과 동일한 `rnn_forward` 및 `rnn_backward`와 같은 함수에 액세스 할 수 있습니다.

In [ ]:
import numpy as np
from utils import *
import random
import pprint

## 1 - Problem Statement

### 1.1 - Dataset and Preprocessing

다음 셀을 실행하여 공룡 이름 데이터 세트를 읽고 고유 문자 (예 : a-z) 목록을 만들고 데이터 세트와 어휘 크기를 계산합니다.

In [ ]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

- 데이터 세트에 나타나는 문자는 알파벳 a-z(26 자)와 개행문자 "\n"입니다.
- 이번 과제에서 개행문자 "\n"은 우리가 강의에서 논의한 `<EOS>`(또는 "문의 끝") 토큰과 유사한 역할을 합니다.
  - 여기서 "\n"은 문장의 끝이 아닌 공룡 이름의 끝을 나타냅니다.
- `char_to_ix` : 아래 셀에서 각 문자를 0-26의 인덱스에 매핑하는 파이썬 딕셔너리 (즉, 해시 테이블)을 만듭니다.
- `ix_to_char` : 각 인덱스를 해당 문자에 다시 매핑하는 두 번째 파이썬 딕셔너리도 만듭니다.
  - 이것은 softmax 레이어의 확률 분포 출력에서 어떤 문자에 해당하는 인덱스를 파악하는 데 도움이됩니다.

In [ ]:
chars = sorted(chars)
print(chars)

In [ ]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

### 1.2 - Overview of the model

모델의 구조는 다음과 같습니다.

- 파라미터 초기화
- 최적화 루프 실행
  - 손실 함수를 계산하기위한 forward propagation
  - 손실 함수에 대한 기울기를 계산하기 위한 back propagation
  - Gradient exploding이 일어나지 않도록 그라디언트를 자릅니다.
  - 경사하강법으로 파라미터를 업데이트합니다.
- 최종 학습 된 파라미터 반환

<img src="arts/rnn.png" style="width:450;height:300px;">
<center>그림 1 : 이전 과제 "Building a Recurrent Neural Network-Step by Step"에서 구현 한 것과 유사한 Recurrent Neural Network</center>

* 각 time step에서 RNN은 이전 문자가 주어지면 다음 문자가 무엇인지 예측하려고합니다.
* 데이터 세트 $ \mathbf {X} = (x ^ {\langle 1 \rangle}, x ^ {\langle 2 \rangle}, ..., x ^{\langle T_x \rangle}) $은 훈련 세트의 문자.
* $ \mathbf {Y} = (y ^ {\langle 1 \rangle}, y ^ {\langle 2 \rangle}, ..., y ^ {\langle T_x \rangle}) $은 동일한 문자 목록입니다. 하지만 한 문자 앞으로 이동했습니다.
* 모든 시간 단계에서 $ t $, $ y ^ {\langle t \rangle} = x ^ {\langle t + 1 \rangle} $. 시간 $ t $에서의 예측은 시간 $ t + 1 $에서의 입력과 동일합니다.


## 2 - Building blocks of the model

이 부분에서는 전체 모델의 두 가지 중요한 블록을 작성합니다.
- Gradient Cliping : Exploding gradient 방지
- Sampling : 캐릭터 생성에 사용되는 기술

그런 다음 이 두 기능을 적용하여 모델을 빌드합니다.

### 2.1 - Clipping the gradients in the optimization loop

#### Exploding gradients

* 계산되는 gradient가 매우 크면 "그라디언트 폭발"이라고합니다.
* 폭발 기울기는 업데이트되는 파라미터가 너무 커서 역전파 동안 최적의 값을 넘어갈 수 있기 때문에 모델 훈련을 더 어렵게 만듭니다.

전체 루프 구조는 일반적으로 다음으로 구성됩니다.
* Forward propagation
* Compute cost
* Backward propagation
* Parameter update

파라미터를 업데이트하기 전에 Gradient clipping을 수행하여 그라디언트가 "폭발"하지 않는지 확인합니다.


#### gradient clipping

아래 연습 문제에서는 그라디언트 딕셔너리를 가져와 클리핑 된 새로운 그라디언트를 반환하는 함수 `clip`을 구현합니다.

* 그라디언트를 자르는 방법에는 여러 가지가 있습니다.
* 간단히 행렬의 요소 별로 클리핑 절차를 사용합니다. 여기서 그라디언트 벡터의 모든 요소는 [-N, N] 범위 사이에 놓 이도록 클리핑됩니다.
* 예를 들어 N = 10 인 경우
  - 범위는 [-10, 10]입니다.
  - 그래디언트 벡터의 구성 요소가 10보다 크면 10으로 설정됩니다.
  - 그래디언트 벡터의 구성 요소가 -10보다 작 으면 -10으로 설정됩니다.
  - 구성 요소가 -10에서 10 사이이면 원래 값을 유지합니다.

<img src="arts/clip.png" style="width:400;height:150px;">
번역 결과
<center>그림 2 : 네트워크가 "Gradient Exploding" 문제에 직면 한 경우 Gradient clipping이 있거나 없는 경우의 경사 하강법 시각화.</center>


**연습 문제**:
딕셔너리 '그라디언트'의 clipping된 그라디언트를 반환하려면 아래 함수를 구현하세요.

* 함수는 최대 임계 값을 인자로 받고 그라디언트의 클리핑 된 버전을 반환합니다.
* [numpy.clip](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html)을 확인하세요.
  - "`out = ...`" 파라미터를 사용해야합니다.
  - "`out`" 파라미터를 사용하면 "in-place" 변수를 업데이트 할 수 있습니다.
  - "`out`" 파라미터를 사용하지 않으면 잘린 변수가 "gradient"변수에 저장되지만 그래디언트 변수 `dWax`,`dWaa`,`dWya`,`db`,`dby`는 업데이트되지 않습니다.

In [ ]:
### GRADED FUNCTION: clip

def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    ### START CODE HERE ###
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in [None]:
        None
    ### END CODE HERE ###
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [ ]:
# Test with a maxvalue of 10
mValue = 10
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, mValue)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

**모범 답안**

```Python
gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 10.]
gradients["dby"][1] = [ 8.45833407]
```

In [ ]:
# Test with a maxValue of 5
mValue = 5
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, mValue)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
del mValue # avoid common issue

**모범 답안 :**
```Python
gradients["dWaa"][1][2] = 5.0
gradients["dWax"][3][1] = -5.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 5.]
gradients["dby"][1] = [ 5.]
```

### 2.2 - Sampling

이제 모델이 학습되었다고 가정해봅시다. 새 텍스트(문자열)를 생성하려고 합니다. 생성 과정은 아래 그림에 설명되어 있습니다.

<img src="arts/dinos3.png" style="width:500;height:300px;">
<center>그림 3 : 이 그림에서는 모델이 이미 훈련되었다고 가정합니다. 첫 번째 단계에서 $ x ^ {\langle 1 \rangle} = \vec{0} $를 전달하고 신경망에서 한 번에 한 문자 씩 샘플링합니다.</center>


**연습 문제**: 문자열을 샘플링하는 `sample` 함수를 구현해보세요. 4가지 단계를 따라 구현합니다.


- **1단계** : 0으로 구성된 "더미" 벡터 $ x ^ {\langle 1 \rangle} = \vec{0} $를 입력합니다.
  - 이것은 문자를 생성하기 전의 기본 입력입니다. 마찬가지로 $ a^{\langle 0 \rangle} = \vec {0} $ 도 설정했습니다.

- **2단계** : 
$ a ^ {\langle 1 \rangle} $ 및 $ \hat {y} ^ {\langle 1 \rangle} $를 얻기 위해 한 단계의 forward propagation을 실행합니다. 공식은 다음과 같습니다.

  - hidden state:  
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t+1 \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

  - activation:
$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

  - prediction:
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$


- $ \hat {y} ^ {\langle t + 1 \rangle} $에 대한 세부 정보 :
  - $ \hat {y} ^ {\langle t + 1 \rangle} $은 (소프트 맥스) 확률 벡터입니다 (항목은 0과 1 사이이고 합은 1입니다).
  - $ \hat {y} ^ {\langle t + 1 \rangle} _i $는 "i"로 인덱싱 된 문자가 다음 문자일 확률을 나타냅니다.
  - 사전 제공되는 `softmax ()`함수를 사용하세요.

#### 추가 힌트

- $ x ^ {\langle 1 \rangle} $은 코드에서 `x`입니다. one-hot 인코딩 벡터를 만들 때 행 수는 고유 문자 수와 같고 열 수는 1과 같은 0으로 구성된 numpy 배열을 만듭니다. 1D 배열이 아니라 2D입니다.
- $ a ^ {\langle 0 \rangle} $은 코드에서 `a_prev`입니다. 행 수는 $ n_{a} $이고 열 수는 1 인 0으로 구성된 numpy 배열입니다. 이는 2D 배열이기도합니다. $ n_{a} $는 $ W_{aa} $의 열 수를 가져 와서 구할 수 있습니다 (행렬 곱셈 $ W_{aa} a ^ {\langle t \rangle} $이 작동하려면 숫자가 일치해야합니다. .
-[numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)
-[numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)

#### Using 2D arrays instead of 1D arrays

- $ x ^ {\langle 1 \rangle} $ 및 $ a ^ {\langle 0 \rangle} $이 1D 벡터가 아니라 2D 배열임을 강조하는 이유가 궁금 할 것입니다.
- numpy의 행렬 곱셈의 경우 2D 행렬에 1D 벡터를 곱하면 결국 1D 배열이됩니다.
- 이것은 우리가 동일한 shape를 가질 것으로 예상했던 두 개의 배열을 추가 할 때 문제가 됩니다.
- 차원 수가 다른 두 배열이 함께 추가되면 Python에선 broadcast 기능이 동작합니다.
- 다음은 1D 배열과 2D 배열 사용의 차이점을 보여주는 몇 가지 샘플 코드입니다.

In [ ]:
matrix1 = np.array([[1,1],[2,2],[3,3]]) # (3,2)
matrix2 = np.array([[0],[0],[0]]) # (3,1) 
vector1D = np.array([1,1]) # (2,) 
vector2D = np.array([[1],[1]]) # (2,1)
print("matrix1 \n", matrix1,"\n")
print("matrix2 \n", matrix2,"\n")
print("vector1D \n", vector1D,"\n")
print("vector2D \n", vector2D)

In [ ]:
print("Multiply 2D and 1D arrays: result is a 1D array\n", 
      np.dot(matrix1,vector1D))
print("Multiply 2D and 2D arrays: result is a 2D array\n", 
      np.dot(matrix1,vector2D))

In [ ]:
print("Adding (3 x 1) vector to a (3 x 1) vector is a (3 x 1) vector\n",
      "This is what we want here!\n", 
      np.dot(matrix1,vector2D) + matrix2)

In [ ]:
print("Adding a (3,) vector to a (3 x 1) vector\n",
      "broadcasts the 1D array across the second dimension\n",
      "Not what we want here!\n",
      np.dot(matrix1,vector1D) + matrix2
     )

- **3단계**: 샘플링:
  - 이제 $ y ^ {\langle t + 1 \rangle} $가 있으므로 공룡 이름의 다음 문자를 선택하려고 합니다. 가장 가능성이 높은 것을 선택하면 모델은 항상 시작 문자가 주어지면 동일한 결과를 생성합니다. 결과를 더 흥미롭게 만들기 위해 np.random.choice를 사용하여 *가능성 이 높지만* 항상 같지는 않은 다음 문자를 선택합니다.
  - $ \hat {y} ^ {\langle t + 1 \rangle} $로 지정된 확률 분포에 따라 다음 캐릭터의 **인덱스**를 선택합니다.
  - 이는 $ \hat {y} ^ {\langle t + 1 \rangle} _i = 0.16 $이면 16 % 확률로 인덱스 "i"를 선택한다는 의미입니다.
  - [np.random.choice] (https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)를 사용합니다.
  `np.random.choice()` 사용 방법의 예 :
  ```python
    np.random.seed (0)
    probs = np.array ([0.1, 0.0, 0.7, 0.2])
    idx = np.random.choice (range (len ((probs)), p = probs)
  ```
  - 이는 분포에 따라 index (`idx`)을 선택한다는 의미입니다.
  $P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.
  - 'p'로 설정된 값은 1D 벡터로 설정되어야합니다.
  - 또한 코드에서`y` 인 $ \ hat {y} ^ {\langle t + 1 \rangle} $는 2D 배열입니다.
  - 또한 구현에서 `np.random.choice`에 대한 첫 번째 인자는 정렬 된 목록 [0,1, .., vocab_len-1] 이지만 `char_to_ix.values​​()`를 사용하는 것은 *적절하지 않습니다*. 파이썬 딕셔너리의 `values​​()` 함수 호출에 의해 반환 된 값의 *순서*는 딕셔너리에 추가되는 순서와 동일한 순서입니다. 채점자는 루틴을 실행할 때 노트북에서 실행할 때와 다른 순서를 가질 수 있습니다.

##### 추가 힌트
- [range](https://docs.python.org/3/library/functions.html#func-range)
- [numpy.ravel](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html)은 다차원 배열을 가져 와서 1D 벡터 내부에 그 내용을 반환합니다.
```python
arr = np.array ([[1,2], [3,4]])
print ( "arr")
print (arr)
print ( "arr.ravel ()")
print (arr.ravel ())
```
Output:
```python
arr
[[1 2]
 [3 4]]
arr.ravel ()
[1 2 3 4]
```

- `append`는 "in-place"작업입니다. 즉, 다음과 같이 코드를 작성하지 마십시오.
```python
fun_hobbies = fun_hobbies.append ( 'learning') ## Doesn't give you what you want
```


- **4단계** : $ x ^ {\langle t \rangle} $로 업데이트
  - `sample()`에서 구현하는 마지막 단계는 현재 $ x ^ {\langle t \rangle} $을 저장하고 있는 변수 `x`를 $ x ^ {\langle t + 1 \rangle}$ 값으로 업데이트하는 것입니다.
  - 예측 결과로 선택한 캐릭터에 상응하는 one-hot 벡터를 생성하여 $ x ^ {\langle t + 1 \rangle} $을 나타냅니다.
  - 그런 다음 1단계에서 $ x ^ {\langle t + 1 \rangle}$ 을 전달하고 생성한 공룡 이름의 끝에 도달했음을 나타내는 "\n"문자를 얻을 때까지 프로세스를 계속 반복합니다.

##### 추가 힌트
- 새로운 one-hot 벡터로 설정하기 전에 `x` 를 재설정하려면 모든 값을 0으로 설정해야합니다.
  - 새 numpy 배열을 만들 수 있습니다. [numpy.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)
  - 또는 하나의 숫자로 모든 값 채우기 : [numpy.ndarray.fill](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.fill.html)

In [ ]:
# GRADED FUNCTION: sample

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the a zero vector x that can be used as the one-hot vector 
    # representing the first character (initializing the sequence generation). (≈1 line)
    x = None
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = None
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # idx is the index of the one-hot vector x that is set to 1
    # All other positions in x are zero.
    # We will initialize idx to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step:
    # sample a character from a probability distribution 
    # and append its index (`idx`) to the list "indices". 
    # We'll stop if we reach 50 characters 
    # (which should be very unlikely with a well trained model).
    # Setting the maximum number of characters helps with debugging and prevents infinite loops. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = None
        z = None
        y = None
        
        # for grading purposes
        np.random.seed(counter+seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        # (see additional hints above)
        idx = None

        # Append the index to "indices"
        None
        
        # Step 4: Overwrite the input x with one that corresponds to the sampled index `idx`.
        # (see additional hints above)
        x = None
        x[None] = None
        
        # Update "a_prev" to be "a"
        a_prev = None
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [ ]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}


indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:\n", indices)
print("list of sampled characters:\n", [ix_to_char[i] for i in indices])

**모범 답안:**

```Python
Sampling:
list of sampled indices:
 [12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 17, 24, 12, 13, 24, 0]
list of sampled characters:
 ['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'q', 'x', 'l', 'm', 'x', '\n']
```

## 3 - Building the language model 

텍스트 생성을 위한 문자 수준 언어 모델을 구축해 봅니다.

### 3.1 - Gradient descent 

* 이 섹션에서는 확률 적 경사 하강법(clipped gradient 포함)의 한 단계를 수행하는 함수를 구현합니다.
* 학습 예제를 한 번에 하나씩 진행하므로 최적화 알고리즘은 확률 적 경사 하강 법이됩니다.

다음은 RNN에 대한 일반적인 최적화 루프의 단계입니다.

- 손실을 계산하기 위해 RNN을 통해 forward propagation
- backward propagation을 수행하여 파라미터에 대한 손실의 기울기를 계산합니다.
- Gradient clipping
- 경사 하강 법을 사용하여 파라미터 업데이트


**연습 문제** : 최적화 프로세스를 구현합니다 (확률 적 경사 하강 법의 한 단계).

다음과 같은 함수가 제공됩니다.

```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```

또한 여러분이 사전에 `clip` 함수를 구현했다는 것을 기억하세요.

#### Parameters

* `parameters`가 `optimize` 함수의 반환 된 값 중 하나가 아니더라도`parameters` 사전 내부의 가중치 및 bias는 최적화에 의해 업데이트됩니다. `parameters` 사전은 참조를 통해 함수에 전달되므로이 사전을 변경하면 함수 외부에서 액세스하더라도 `parameters` 사전이 변경됩니다.
* 파이썬 사전과 목록은 "참조에 의한 전달"입니다. 즉, 딕셔너리 변수를 함수에 전달하고 함수 내에서 사전을 수정하면 동일한 딕셔너리가 변경됩니다 (해당 딕셔너리의 복사본이 아님).

In [ ]:
# GRADED FUNCTION: optimize

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    ### START CODE HERE ###
    
    # Forward propagate through time (≈1 line)
    loss, cache = None
    
    # Backpropagate through time (≈1 line)
    gradients, a = None
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = None
    
    # Update parameters (≈1 line)
    parameters = None
    
    ### END CODE HERE ###
    
    return loss, gradients, a[len(X)-1]

In [ ]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

**모범 답안:**

```Python
Loss = 126.503975722
gradients["dWaa"][1][2] = 0.194709315347
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [ 0.01538192]
a_last[4] = [-1.]
```

### 3.2 - Training the model 

* 공룡 이름의 데이터 세트가 주어지면 데이터 세트의 각 line (이름 하나)을 하나의 훈련 데이터로 사용합니다.
* 확률 적 경사 하강 법의 2000 단계마다 무작위로 선택한 여러 이름을 샘플링하여 알고리즘이 어떻게 작동하는지 확인합니다.


**연습 문제** : 안내에 따라 `model()`을 구현하세요. `examples[index]`에 공룡 이름 (문자열)이 하나 포함 된 경우 예제 (X, Y)를 만들려면 다음을 사용할 수 있습니다.

##### Set the index `idx` into the list of examples


* 반복문을 사용하여 "examples" 리스트에서 공룡 이름을 섞은 목록을 살펴 봅니다.
* 예를 들어, n_e 예제가 있고 for 루프가 인덱스를 n_e 이후로 증가시키는 경우, j가 n_e 일 때 모델에 예제를 계속 공급할 수 있도록 인덱스주기를 0으로 되 돌리는 방법을 생각해보십시오. , n_e + 1 등
* 힌트 : n_e + 1을 n_e로 나눈 값은 0이고 나머지는 1입니다.
* `%` 는 파이썬의 나머지 연산자입니다.

##### Extract a single example from the list of examples

* `single_example` : 이전에 설정한 `idx` 인덱스를 사용하여 예제 목록에서 한 단어를 가져옵니다.

##### Convert a string into a list of characters: `single_example_chars`


* `single_example_chars` : 문자열은 문자 목록입니다.
* list conprehension(for 루프보다 권장 됨)을 사용하여 문자 목록을 생성 할 수 있습니다.
```Python
str = 'I love learning'
list_of_chars = [c for c in str]
print(list_of_chars)
```

```
[ 'I', '', 'l', 'o', 'v', 'e', ​​'', 'l', 'e', ​​'a', 'r', 'n', 'i' , 'n', 'g']
```

##### Convert list of characters to a list of integers: `single_example_ix`

* 각 문자와 관련된 인덱스가 포함 된 목록을 만듭니다.
* 딕셔너리 `char_to_ix`를 사용합니다.
* 문자열에서 문자 목록을 가져 오는 데 사용되는 list comprehension과 결합 할 수 있습니다.

##### Create the list of input characters: `X`

* `rnn_forward`는 **`None`** 값을 플래그로 사용하여 입력 벡터를 제로 벡터로 설정합니다.
* 입력 문자 목록 앞에 리스트 [**`None`**]을 추가합니다.
* 리스트에 값을 추가하는 방법은 여러 가지가 있습니다. 한 가지 방법은 두 개의 목록을 함께 추가하는 것입니다. `[ 'a'] + [ 'b']`

##### Get the integer representation of the newline character `ix_newline`

* `ix_newline` : 개행 문자는 공룡 이름의 끝을 나타냅니다.
  - 개행 문자 `'\n'`의 정수 표현을 얻습니다.
  - `char_to_ix` 사용

##### Set the list of labels (integer representation of the characters): `Y`

* 목표는 공룡 이름의 다음 문자를 예측하도록 RNN을 훈련시키는 것이므로 레이블은 입력 'X'의 문자보다 한 단계 앞선 문자 목록입니다.
  - 예를 들어`Y [0]`은`X [1]`과 동일한 값을 포함합니다.
* RNN은 마지막 문자에서 줄 바꿈을 예측해야하므로 레이블 끝에 ix_newline을 추가합니다.
  - 줄 바꿈 문자의 정수 표현을`Y` 끝에 추가합니다.
  - '추가'는 내부 작업입니다.
  - 두 개의 목록을 함께 추가하는 것이 더 쉬울 수 있습니다.

In [ ]:
# GRADED FUNCTION: model

def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27, verbose = False):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text (size of the vocabulary)
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        ### START CODE HERE ###
        
        # Set the index `idx` (see instructions above)
        idx = None
        
        # Set the input X (see instructions above)
        single_example = None
        single_example_chars = None
        single_example_ix = None
        X = None
        
        # Set the labels Y (see instructions above)
        ix_newline = None
        Y = None

        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = None
        
        ### END CODE HERE ###
        
        # debug statements to aid in correctly forming X, Y
        if verbose and j in [0, len(examples) -1, len(examples)]:
            print("j = " , j, "idx = ", idx,) 
        if verbose and j in [0]:
            print("single_example =", single_example)
            print("single_example_chars", single_example_chars)
            print("single_example_ix", single_example_ix)
            print(" X = ", X, "\n", "Y =       ", Y, "\n")
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result (for grading purposes), increment the seed by one. 
      
            print('\n')
        
    return parameters

다음 셀을 실행하면 첫 번째 반복에서 모델이 임의의 문자를 출력하는 것을 관찰해야합니다. 수천 번의 반복 후에 모델은 합리적으로 보이는 이름을 생성하는 방법을 배워야합니다.

In [ ]:
parameters = model(data, ix_to_char, char_to_ix, verbose = True)

**모범 답안**

```Python
j =  0 idx =  0
single_example = turiasaurus
single_example_chars ['t', 'u', 'r', 'i', 'a', 's', 'a', 'u', 'r', 'u', 's']
single_example_ix [20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19]
 X =  [None, 20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19] 
 Y =        [20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19, 0] 

Iteration: 0, Loss: 23.087336

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


j =  1535 idx =  1535
j =  1536 idx =  0
Iteration: 2000, Loss: 27.884160

...

Iteration: 34000, Loss: 22.447230

Onyxipaledisons
Kiabaeropa
Lussiamang
Pacaeptabalsaurus
Xosalong
Eiacoteg
Troia
```


## 결론

알고리즘이 훈련이 끝날 무렵 그럴듯한 공룡 이름을 생성하기 시작했음을 알 수 있습니다. 처음에는 임의의 캐릭터를 생성했지만 마지막에는 멋진 엔딩으로 공룡 이름을 볼 수있었습니다. 알고리즘을 더 오래 실행하고 하이퍼 파라미터를 사용하여 더 나은 결과를 얻을 수 있는지 확인하십시오. 우리의 구현은 `maconucon`, `marloralus` 및 `macingsersaurus`와 같은 정말 멋진 이름을 생성했습니다. 여러분의 모델은 공룡 이름이 'saurus', 'don', 'aura', 'tor'등으로 끝나는 경향이 있다는 것을 알게되기를 바랍니다.

모델이 멋지지 않은 이름을 생성하는 경우 그것이 완전히 모델의 탓만은 아닙니다. 실제 공룡 이름이 모두 멋지게 들리는 것은 아닙니다. (예를 들어, `dromaeosauroides`는 실제 공룡 이름이며 훈련 세트에 있습니다.)하지만이 모델은 가장 멋진 것을 선택할 수있는 후보 세트를 제공해야합니다!

이 할당은 비교적 작은 데이터 세트를 사용했기 때문에 CPU에서 RNN을 빠르게 훈련 할 수있었습니다. 영어 모델을 학습하려면 훨씬 더 큰 데이터 세트가 필요하며 일반적으로 훨씬 더 많은 계산이 필요하며 GPU에서 여러 시간 동안 실행할 수 있습니다. 우리는 공룡 이름을 꽤 오랫동안 사용했으며 지금까지 우리가 가장 좋아하는 이름은 위대하고 무패하며 치열한 망고 사우루스입니다!

<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">

## 4 - Writing like Shakespeare

이 노트의 나머지 부분은 선택 사항이며 채점되지 않지만 재미 있고 유익한 내용이므로 어쨌든 해주시기를 바랍니다.

비슷하지만 더 복잡한 작업은 셰익스피어의 시를 만드는 것입니다. 공룡 이름 데이터 세트에서 배우는 대신 셰익스피어의 시 모음을 사용할 수 있습니다. LSTM 셀을 사용하면 텍스트의 여러 문자에 걸친 장기 종속성을 학습 할 수 있습니다. 예를 들어, 시퀀스 어딘가에 나타나는 문자가 시퀀스에서 훨씬 나중에 다른 문자가되어야하는 것에 영향을 줄 수 있습니다. 이름이 매우 짧기 때문에 이러한 장기적 의존성은 공룡 이름에 덜 중요했습니다.

<img src="arts/shakespeare.jpg" style="width:500;height:400px;">
<center> Let's become poets!</center>


Keras로 셰익스피어시 생성 모델을 구현했습니다. 다음 셀을 실행하여 필요한 패키지 및 모델을 로드하십시오. 이 작업은 몇 분 정도 걸릴 수 있습니다.

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from shakespeare_utils import *
import sys
import io

시간을 절약하기 위해 우리는 이미 [* "The Sonnets"*] (shakespeare.txt)라는 셰익스피어시 모음에 대해 ~ 1000 epochs에 대한 모델을 훈련 시켰습니다.

한 개의 epoch에 대해 추가로 모델을 훈련시켜 봅시다. one epoch에 대한 학습이 완료되려면 몇 분 정도 걸립니다. 입력을 요청하는 `generate_output`을 실행할 수 있습니다 (`<`40 자). 시는 문장으로 시작하고 RNN- 셰익스피어가 나머지시를 완성합니다! 예를 들어, "Forsooth this maketh no sense"(따옴표를 입력하지 마십시오)를 시도하십시오. 끝에 공백을 포함하는지 여부에 따라 결과가 다를 수도 있습니다. 두 가지 방법을 모두 시도하고 다른 입력도 시도해보세요.

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

In [ ]:
# Run this cell to try with different inputs without having to re-train the model 
generate_output()

RNN-Shakespeare 모델은 공룡 이름으로 만든 모델과 매우 유사합니다. 유일한 주요 차이점은 다음과 같습니다.
- 기본 RNN 대신 LSTM을 사용하여 장거리 종속성 캡처
- 모델이 더 깊고 적층 된 LSTM 모델 (2 층)
- Python 대신 Keras를 사용하여 코드 단순화

자세한 내용은 [GitHub](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)에서 Keras Team의 텍스트 생성 구현을 확인할 수도 있습니다.

이 과제를 끝마친 것을 축하합니다!

**References**:
- This exercise took inspiration from Andrej Karpathy's implementation: https://gist.github.com/karpathy/d4dee566867f8291f086. To learn more about text generation, also check out Karpathy's [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).
- For the Shakespearian poem generator, our implementation was based on the implementation of an LSTM text generator by the Keras team: https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py 